# Boss 直聘自动化 - 完整功能演示

本 Notebook 提供 Boss 直聘自动化系统的完整功能演示，包括：

## 🚀 **核心功能**
- **智能简历处理**：多种捕获方法（WASM、Canvas拦截、截图）
- **AI决策系统**：结合岗位要求与简历进行智能匹配
- **批量处理**：支持多候选人并发处理
- **OCR集成**：本地和云端OCR处理
- **通知系统**：DingTalk集成

## 📋 **API功能演示**
- 服务状态检查与健康监控
- 消息列表获取与聊天历史
- 简历请求与在线简历查看
- 多种简历捕获方法演示
- 图像处理与OCR转换
- 批量操作与并发处理

## 🔧 **前置条件**
- Boss服务已运行 (`python start_service.py`)
- 可选环境变量：
  - `OPENAI_API_KEY`：AI决策功能
  - `DINGTALK_WEBHOOK`：通知功能
  - `DEMO_CHAT_ID`：演示用聊天ID


In [1]:
# 📦 导入依赖和初始化客户端
import os
import json
import time
from datetime import datetime
from typing import Dict, List, Optional
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64

from boss_client import BossClient, ResumeResult

# 配置
BASE_URL = os.environ.get('BOSS_SERVICE_URL', 'http://127.0.0.1:5001')
DEMO_CHAT_ID = os.environ.get('DEMO_CHAT_ID', '46232784-0')  # 演示用聊天ID

print("🚀 Boss 直聘自动化系统 - 完整功能演示")
print("=" * 60)

# 初始化客户端
client = BossClient(BASE_URL)

# 检查服务状态
print("🔍 检查服务状态...")
status = client.get_status()
print(f"✅ 服务状态: {status.get('status', 'unknown')}")
print(f"📊 浏览器状态: {status.get('browser_status', 'unknown')}")
print(f"⏰ 启动时间: {status.get('startup_time', 'unknown')}")

# 健康检查
if client.is_service_healthy():
    print("✅ 服务健康检查通过")
else:
    print("❌ 服务健康检查失败")
    raise RuntimeError("服务不可用，请先启动服务 (python start_service.py)")

print("\n🔄 服务重启功能演示...")
# 注意：实际环境中谨慎使用重启功能
# restart_result = client.restart()
# print(f"重启结果: {restart_result}")
DEMO_CHAT_ID

🚀 Boss 直聘自动化系统 - 完整功能演示
🔍 检查服务状态...
✅ 服务状态: running
📊 浏览器状态: unknown
⏰ 启动时间: unknown
✅ 服务健康检查通过

🔄 服务重启功能演示...


'46232784-0'

## 🔧 **基础服务操作**

演示基本的服务管理功能


In [2]:
# 🔧 基础服务操作演示

print("📋 获取通知列表...")
notifications = client.get_notifications(limit=5)
print(f"📊 通知数量: {len(notifications.get('notifications', []))}")
if notifications.get('notifications'):
    print("📝 最新通知:")
    for notif in notifications['notifications'][-3:]:
        print(f"   - {notif.get('message', 'N/A')} ({notif.get('level', 'info')})")

print("✅ 基础服务操作演示完成")


📋 获取通知列表...
📊 通知数量: 5
📝 最新通知:
   - 事件管理器设置成功 (success)
   - 已导航到聊天页面 (info)
   - 持久化浏览器会话启动成功！ (success)
✅ 基础服务操作演示完成


## 💬 **消息管理功能**

演示消息列表获取、聊天历史查看等功能


In [13]:
# 💬 消息管理功能演示

print("📋 获取消息列表...")
messages_response = client.get_messages(limit=10)
print(f"✅ 消息获取成功: {messages_response.get('success')}")
print(f"📊 消息数量: {len(messages_response.get('messages', []))}")

if messages_response.get('success') and messages_response.get('messages'):
    messages = messages_response['messages']
    print(f"\n📝 消息列表 (前5条):")
    for i, msg in enumerate(messages[:5]):
        print(f"   {i+1}. {msg.get('candidate', 'Unknown')} - {msg.get('message', 'N/A')[:50]}...")
        print(f"      Chat ID: {msg.get('chat_id')} | 职位: {msg.get('job_title', 'N/A')}")
    
    # 使用第一个消息进行后续演示
    if messages:
        demo_chat_id = messages[0]['chat_id']
        demo_candidate = messages[0].get('candidate', 'Unknown')
        print(f"\n🎯 选择演示候选人: {demo_candidate} (Chat ID: {demo_chat_id})")
        
        # 获取聊天历史
        print(f"\n📜 获取聊天历史...")
        history_response = client.get_chat_history(demo_chat_id)
        if history_response.get('success'):
            history = history_response.get('messages', [])
            print(f"📊 历史消息数量: {len(history)}")
            if history:
                print(f"📝 最新消息: {history[-1].get('content', 'N/A')[:100]}...")
        else:
            print(f"❌ 获取聊天历史失败: {history_response.get('error')}")
else:
    print("❌ 没有找到消息，请确保有活跃的聊天")
    demo_chat_id = DEMO_CHAT_ID
    print(f"🎯 使用默认Chat ID: {demo_chat_id}")

print("✅ 消息管理功能演示完成")
import random

# 从消息列表中随机选择一个 demo_chat_id（如果有消息）
if messages_response.get('success') and messages_response.get('messages'):
    messages = messages_response['messages']
    if messages:
        demo_chat_id = random.choice(messages)['chat_id']
        print(f"🎲 随机选择的 demo_chat_id: {demo_chat_id}")
    else:
        demo_chat_id = DEMO_CHAT_ID
        print(f"⚠️ 消息列表为空，使用默认 demo_chat_id: {demo_chat_id}")
else:
    demo_chat_id = DEMO_CHAT_ID
    print(f"⚠️ 消息获取失败或无消息，使用默认 demo_chat_id: {demo_chat_id}")


📋 获取消息列表...
✅ 消息获取成功: True
📊 消息数量: 10

📝 消息列表 (前5条):
   1. 张诚 - 你撤回了一条消息...
      Chat ID: 687609714-0 | 职位: 大模型算法工程师
   2. 金久五 - 你撤回了一条消息...
      Chat ID: 643094543-0 | 职位: 大模型算法工程师
   3. 李庆旭 - 占用您百忙之中的一点时间，关注贵公司很久了，和我的职业规划很吻合，想和您聊聊...
      Chat ID: 38187855-0 | 职位: 大模型算法工程师
   4. 于先生 - 你好...
      Chat ID: 689994060-0 | 职位: 大模型算法工程师
   5. 黄耀辉 - 你好，我想应聘贵公司的大模型算法工程师，期盼回复，谢谢！...
      Chat ID: 642482457-0 | 职位: 大模型算法工程师

🎯 选择演示候选人: 张诚 (Chat ID: 687609714-0)

📜 获取聊天历史...
📊 历史消息数量: 0
✅ 消息管理功能演示完成
🎲 随机选择的 demo_chat_id: 83996666-0


## 💬 **发送消息功能**

演示发送文本消息到指定对话


In [10]:
# 发送消息演示
if demo_chat_id:
    print(f"📤 发送消息到 Chat ID: {demo_chat_id}")
    
    # 发送测试消息
    test_message = "您好，我对您的简历很感兴趣，希望能进一步沟通。"
    send_result = client.send_message(demo_chat_id, test_message)
    
    if send_result.get('success'):
        print(f"✅ 消息发送成功: {send_result.get('details')}")
        print(f"📝 发送内容: {test_message}")
    else:
        print(f"❌ 消息发送失败: {send_result.get('error')}")
        
    # 等待一下再获取聊天历史验证
    time.sleep(2)
    print("\n📜 验证消息是否发送成功...")
    history_response = client.get_chat_history(demo_chat_id)
    if history_response.get('success'):
        history = history_response.get('messages', [])
        print(f"📊 更新后历史消息数量: {len(history)}")
        if history:
            latest_message = history[-1]
            print(f"📝 最新消息: {latest_message.get('content', 'N/A')[:100]}...")
    else:
        print(f"❌ 获取聊天历史失败: {history_response.get('error')}")
else:
    print("❌ 没有可用的Chat ID进行消息发送演示")
    
print("✅ 发送消息功能演示完成")


📤 发送消息到 Chat ID: 687609714-0
✅ 消息发送成功: 消息发送成功
📝 发送内容: 您好，我对您的简历很感兴趣，希望能进一步沟通。

📜 验证消息是否发送成功...
📊 更新后历史消息数量: 0
✅ 发送消息功能演示完成


In [18]:
# demo_chat_id = '687609714-0'
print(f"\n🔍 在线简历演示({demo_chat_id})")

print("\n2️⃣ WASM模式捕获...")
result_wasm = client.view_online_resume(demo_chat_id)
print(f"✅ WASM模式结果: {result_wasm.success}")
if not result_wasm.success:
    print(f"❌ WASM模式失败: {result_wasm.details}")


print("✅ 简历处理功能演示完成")
if isinstance(result_wasm.text, str):
    print(result_wasm.text[:200])
else:
    print(result_wasm.text)



🔍 在线简历演示(687609714-0)

2️⃣ WASM模式捕获...
✅ WASM模式结果: False
❌ WASM模式失败: 
✅ 简历处理功能演示完成
None


In [ ]:
print("📋 简历请求功能演示...")
# 请求简历
request_result = client.request_resume(demo_chat_id)
print(f"✅ 简历请求结果: {request_result.get('success')}")
if request_result.get('success'):
    print(f"📝 请求消息: {request_result.get('details', 'N/A')}")
else:
    print(f"❌ 请求失败: {request_result.get('error')}")



📋 简历请求功能演示...
✅ 简历请求结果: True
📝 请求消息: 简历请求已发送（按钮已禁用）


In [14]:
# 查看离线简历演示
demo_chat_id = '643094543-0'
print(f"📄 查看简历 Chat ID: {demo_chat_id}")
    
# 点击查看附件简历
view_result = client.view_resume(demo_chat_id)

if view_result.get('success'):
    print(f"✅ 简历查看器已打开: {view_result.get('details')}")
    print(view_result['content'][:200])
else:
    print(f"❌ 查看简历失败: {view_result.get('error')}")
    print(f"📝 错误详情: {view_result}")
    


📄 查看简历 Chat ID: 643094543-0
✅ 简历查看器已打开: 简历查看器已打开
个人简历

基本信息

姓名： 金久五出生年月： 1991-01

性别： 男电话： 198-6372-0995

邮箱： jwjin200@163.com微信：JJW910106

我是一名统计学研究生，在机器学习、统计建模和优化算法拥有丰富和深厚的研究经验。在研究生学习期间，我在顶级期刊

Annals of Statistics上发表了一篇论文。通过在美国顶级银行Wells Fargo的实习


## 📄 **简历请求功能**

演示简历请求、多种捕获方法、OCR处理等功能


## 🤖 **AI决策系统**

演示结合岗位要求和简历进行智能匹配决策


In [ ]:
# 读取岗位要求 YAML
import yaml, json
with open('jobs/criteria.yaml','r',encoding='utf-8') as f:
    criteria = yaml.safe_load(f)
criteria


{'roles': [{'id': 'default',
   'position': '大模型算法工程师',
   'description': '负责大语言模型/多模态模型的训练、微调与落地；\n参与推理性能优化与工程化部署；\n与业务协作将模型能力嵌入产品。\n',
   'target_profile': '- 计算机/数学/电子工程等相关专业，硕士及以上优先；\n- 熟悉 Transformer/Lora/PEFT 等；\n- 3-8 年算法或相关经验，具备端到端项目经验。\n',
   'filters': {'must_have': ['Python',
     {'深度学习框架': ['PyTorch', 'TensorFlow']},
     'LLM/LMM 相关经验'],
    'nice_to_have': ['C++', '分布式训练(DeepSpeed/Megatron)', '向量数据库/检索增强(RAG)'],
    'must_not': ['仅运维/测试背景', '与岗位无关的实习仅有']},
   'keywords': {'positive': ['大模型',
     'LLM',
     'Transformer',
     'Fine-tuning',
     'LoRA',
     'RLHF',
     '知识蒸馏',
     'RAG'],
    'negative': ['爬虫', '外包', '纯NLP规则']},
   'scoring': {'weights': {'experience': 0.35,
     'skills_match': 0.35,
     'projects': 0.2,
     'education': 0.1},
    'threshold': {'greet': 0.7, 'borderline': 0.6}},
   'notes': '可根据岗位变体微调 filters/keywords/weights。\n'}]}

In [ ]:
# 组合 YAML + 简历，调用 OpenAI 进行匹配决策
role = criteria['roles'][0]
import requests, json
api_key = os.environ.get('OPENAI_API_KEY')
if not api_key:
    raise RuntimeError('缺少 OPENAI_API_KEY')

headers = {'Authorization': f'Bearer {api_key}', 'Content-Type': 'application/json'}
prompt = {
    'role': 'user',
    'content': (
        "你是资深HR。根据以下职位要求(YAML)与候选人简历(Markdown)进行匹配评分，输出JSON："
        "{score: 0-1, decision: 'greet'|'skip'|'borderline', reasons: [..], highlights: [..], risks: [..]}。"
        "职位YAML:\n" + yaml.safe_dump(role, allow_unicode=True) + "\n简历Markdown:\n" + md_text
    )
}
payload = {
    'model': os.environ.get('OPENAI_TEXT_MODEL', 'gpt-4o-mini'),
    'messages': [prompt],
    'response_format': {"type": "json_object"}
}
resp = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=payload, timeout=60)
resp.raise_for_status()
content = resp.json()['choices'][0]['message']['content']
decision = json.loads(content)
decision


RuntimeError: 缺少 OPENAI_API_KEY

In [ ]:
# 可选：若决策为 greet，发送钉钉通知
if isinstance(decision, dict) and decision.get('decision') == 'greet':
    hook = os.environ.get('DINGTALK_WEBHOOK')
    if hook:
        msg = {
            "msgtype": "text",
            "text": {"content": f"建议打招呼: chat_id={CHAT_ID}\nscore={decision.get('score')}\nreasons={decision.get('reasons')}"}
        }
        try:
            requests.post(hook, json=msg, timeout=10)
            print('DingTalk 已通知')
        except Exception as e:
            print('DingTalk 通知失败:', e)
else:
    print('决策不是 greet，跳过通知')


### 推荐牛人列表动作

In [17]:
# 👥 查看推荐候选人
print("🔍 获取推荐候选人列表...")

# 调用推荐候选人接口
result = client.get_recommended_candidates(limit=10)

if result.get('success'):
    candidates = result.get('candidates', [])
    print(f"✅ 成功获取 {len(candidates)} 个推荐候选人")
    print()
    
    # 显示候选人信息
    for i, candidate in enumerate(candidates, 1):
        print(f"📋 候选人 {i}:")
        print(f"   👤 姓名: {candidate.get('name', 'N/A')}")
        print(f"   💼 职位: {candidate.get('job_title', 'N/A')}")
        print(f"   🏢 公司: {candidate.get('company', 'N/A')}")
        print(f"   💰 薪资: {candidate.get('salary', 'N/A')}")
        print(f"   📍 地点: {candidate.get('location', 'N/A')}")
        print(f"   ⏳ 经验: {candidate.get('experience', 'N/A')}")
        print(f"   🎓 学历: {candidate.get('education', 'N/A')}")
        
        if candidate.get('tags'):
            print(f"   🏷️  标签: {', '.join(candidate['tags'])}")
        
        if candidate.get('link'):
            print(f"   🔗 链接: {candidate['link']}")
        
        print()
else:
    print(f"❌ 获取推荐候选人失败: {result}")
    

🔍 获取推荐候选人列表...
❌ 获取推荐候选人失败: {'success': False, 'error': 'Request timeout'}


## 🗑️ **丢弃候选人功能**

演示丢弃候选人的功能 - 点击"不合适"按钮


In [ ]:
# 丢弃候选人演示
demo_chat_id = '643094543-0'  # 使用一个测试用的Chat ID
print(f"🗑️ 丢弃候选人 Chat ID: {demo_chat_id}")

# 丢弃候选人
discard_result = client.discard_candidate(demo_chat_id)

if discard_result.get('success'):
    print(f"✅ 候选人已丢弃: {discard_result.get('details')}")
    print(f"📝 操作时间: {discard_result.get('timestamp')}")
else:
    print(f"❌ 丢弃候选人失败: {discard_result.get('error')}")
    print(f"📝 错误详情: {discard_result.get('details')}")

print("✅ 丢弃候选人功能演示完成")
